In [13]:
import __init__
#
from IPython.display import HTML, display
import pandas as pd
import numpy as np
import statsmodels.api as sm
#
from community_analysis import MIN_PICKUP_RATIO
#
def text_display(text, font_size):
    display(HTML('<font size=%d>' % font_size + text + '</font>'))   

In [14]:
def regression(dv, df):
    oc_dv = 'roamingTime' if dv == 'spendingTime' else 'spendingTime'
    rdf = df.copy(deep=True).drop([oc_dv], axis=1)
    candi_dummies = []
    num_iter = 1
    while True:
        for i, vs in enumerate(zip(*rdf.values)):
            if rdf.columns[i] == dv:
                continue
            if sum(vs) > len(rdf) * MIN_PICKUP_RATIO * num_iter:
                candi_dummies.append(rdf.columns[i])
        if len(rdf) <= len(candi_dummies):
            candi_dummies = []
            num_iter += 1
        else:
            break
    y = rdf[dv]
    X = rdf[candi_dummies]
    X = sm.add_constant(X)
    return sm.OLS(y, X, missing='drop').fit()

In [15]:
from community_analysis import tfZ_TP_dpath, tfZ_TP_prefix
#
did1 = 12327
reducerID = 2265 % 60
df = pd.read_csv('%s/%s2012-%d.csv' % (tfZ_TP_dpath, tfZ_TP_prefix, reducerID))

In [16]:
df.head()

,month,day,timeFrame,zi,zj,tfZ,did,spendingTime,roamingTime,1,...,32764,32746,32748,32753,32755,32759,32762,65532,32766,65535
0,1,2,14,53,29,"(14,53,29)",24860,411,656,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,14,54,19,"(14,54,19)",29989,16,47,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,14,50,11,"(14,50,11)",19652,389,603,0,...,0,0,0,0,0,0,0,0,0,0
3,1,2,14,30,34,"(14,30,34)",12920,122,455,0,...,0,0,0,0,0,0,0,0,0,0
4,1,2,14,65,15,"(14,65,15)",13701,362,788,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
text = '# of drivers in a file: %d' % len(set(df['did']))
text_display(text, 5)

In [18]:
did1_df = df[(df['did'] == did1)].copy(deep=True)
did1_df = did1_df.drop(['month', 'day', 'timeFrame', 'zi', 'zj', 'tfZ', 'did'], axis=1)
if '%d' % did1 in did1_df.columns:
    did1_df = did1_df.drop(['%d' % did1], axis=1)

In [19]:
text = '# of observations: %d' % len(did1_df)
text_display(text, 5)
text = '# of indepedent variables:: %d'% len(did1_df.columns)
text_display(text, 5)

In [20]:
text = 'Average: spendingTime %.2f, roamingTime %.2f' % (did1_df['spendingTime'].mean(), did1_df['roamingTime'].mean())
text_display(text, 5)

In [21]:
SP_res = regression('spendingTime', did1_df)
print SP_res.summary()

                            OLS Regression Results                            
Dep. Variable:           spendingTime   R-squared:                       0.518
Model:                            OLS   Adj. R-squared:                  0.236
Method:                 Least Squares   F-statistic:                     1.837
Date:                Sat, 11 Feb 2017   Prob (F-statistic):            0.00225
Time:                        13:30:30   Log-Likelihood:                -1104.3
No. Observations:                 180   AIC:                             2343.
Df Residuals:                     113   BIC:                             2556.
Df Model:                          66                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         91.0223     18.515      4.916      0.0

In [22]:
significant_drivers = set()
for _did0, pv in SP_res.pvalues.iteritems():
    if _did0 == 'const':
        continue
    if pv < 0.01:
        significant_drivers.add(_did0)
positive_ef_drivers = set()
for _did0, cof in SP_res.params.iteritems():
    if _did0 == 'const':
        continue
    if cof > 0:
        positive_ef_drivers.add(_did0)

In [23]:
for _did0 in significant_drivers.difference(positive_ef_drivers):
    print _did0, did1, SP_res.params[_did0]

33477 12327 -131.633516904
3426 12327 -109.748547504


In [24]:
import sys
for p in sys.path:
    print p


/Library/Frameworks/Python.framework/Versions/2.7/lib/python27.zip
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/plat-darwin
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/plat-mac
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/plat-mac/lib-scriptpackages
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/lib-tk
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/lib-old
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/lib-dynload
/Users/JerryHan88/Library/Python/2.7/lib/python/site-packages
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/statsmodels-0.8.0rc1-py2.7-macosx-10.6-intel.egg
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/patsy-0.4.1-py2.7.egg
/Library/Frameworks/Python.framework/Vers

In [29]:
from community_analysis import SIGINIFICANCE_LEVEL, MIN_PICKUP_RATIO
def regression(dv, df):
    oc_dv = 'roamingTime' if dv == 'spendingTime' else 'spendingTime'
    rdf = df.copy(deep=True).drop([oc_dv], axis=1)
    rdf = rdf[~(np.abs(rdf[dv] - rdf[dv].mean()) > (3 * rdf[dv].std()))]
    candi_dummies = []
    num_iter = 1
    while True:
        for i, vs in enumerate(zip(*rdf.values)):
            if rdf.columns[i] == dv:
                continue
            if sum(vs) > len(rdf) * MIN_PICKUP_RATIO * num_iter:
                candi_dummies.append(rdf.columns[i])
        if len(rdf) <= len(candi_dummies):
            candi_dummies = []
            num_iter += 1
        else:
            break
    y = rdf[dv]
    X = rdf[candi_dummies]
    X = sm.add_constant(X)
    return sm.OLS(y, X, missing='drop').fit()

reducerID = 2265 % 60
fpath = '%s/%s%s-%d.csv'% (tfZ_TP_dpath, tfZ_TP_prefix,'2012', reducerID)
df = pd.read_csv(fpath)

SP_graph = {}


# for i, did1 in enumerate(set(df['did'])):
did1 = 12327
#    
did1_df = df[(df['did'] == did1)].copy(deep=True)
did1_df = did1_df.drop(['month', 'day', 'timeFrame', 'zi', 'zj', 'tfZ', 'did'], axis=1)
if '%d' % did1 in did1_df.columns:
    did1_df = did1_df.drop(['%d' % did1], axis=1)
#
SP_res = regression('spendingTime', did1_df)
if SP_res.f_pvalue < SIGINIFICANCE_LEVEL:
    significant_drivers = set()
    for _did0, pv in SP_res.pvalues.iteritems():
        if _did0 == 'const':
            continue
        if pv < SIGINIFICANCE_LEVEL:
            significant_drivers.add(_did0)
    positive_ef_drivers = set()
    for _did0, cof in SP_res.params.iteritems():
        if _did0 == 'const':
            continue
        if cof > 0:
            positive_ef_drivers.add(_did0)
    for _did0 in significant_drivers.difference(positive_ef_drivers):
        print (int(_did0), did1), SP_res.params[_did0]
        SP_graph[int(_did0), did1] = SP_res.params[_did0]

print SP_graph

(33477, 12327) -89.1434337632
{(33477, 12327): -89.143433763234313}


In [31]:
print SP_res.summary()

                            OLS Regression Results                            
Dep. Variable:           spendingTime   R-squared:                       0.571
Model:                            OLS   Adj. R-squared:                  0.242
Method:                 Least Squares   F-statistic:                     1.735
Date:                Sat, 11 Feb 2017   Prob (F-statistic):            0.00508
Time:                        13:39:55   Log-Likelihood:                -994.16
No. Observations:                 176   AIC:                             2142.
Df Residuals:                      99   BIC:                             2386.
Df Model:                          76                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         79.2369     12.752      6.214      0.0

In [22]:
RP_res = regression('roamingTime', did1_df)
print RP_res.summary()

                            OLS Regression Results                            
Dep. Variable:            roamingTime   R-squared:                       0.252
Model:                            OLS   Adj. R-squared:                  0.085
Method:                 Least Squares   F-statistic:                     1.506
Date:                Thu, 19 Jan 2017   Prob (F-statistic):           2.75e-05
Time:                        10:40:19   Log-Likelihood:                -8566.7
No. Observations:                1181   AIC:                         1.757e+04
Df Residuals:                     964   BIC:                         1.867e+04
Df Model:                         216                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        460.0623     18.669     24.643      0.0

In [23]:
header = ['D.V.', 'SP coef.', 'RP coef.', 'SP pv.', 'RP pv.']
df_data = {k: [] for k in header}
for dv, sp_pv in SP_res.pvalues.iteritems():
    for i, v in enumerate([dv, SP_res.params[dv], RP_res.params[dv], sp_pv, RP_res.pvalues[dv]]):
        if sp_pv < 0.01 and RP_res.pvalues[dv] < 0.01 :
            df_data[header[i]].append(v)
df = pd.DataFrame(df_data)[header]
df

,D.V.,SP coef.,RP coef.,SP pv.,RP pv.
0,const,102.507085,460.062324,2.631940e-26,2.216947e-104
